# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = "../WeatherPy/output_data/cities.csv"
weather_file_df = pd.read_csv(weather_file)
weather_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,54.72,82,36,1.01,AU,1624067686
1,Rikitea,-23.1203,-134.9692,74.84,77,8,23.78,PF,1624067580
2,Port Hawkesbury,45.6169,-61.3485,64.35,79,77,1.01,CA,1624067688
3,Alyangula,-13.8483,136.4192,76.87,57,100,11.50,AU,1624067688
4,Puerto Ayora,-0.7393,-90.3518,72.55,96,47,4.00,EC,1624067587


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_file_df[["Lat", "Lng"]].astype(float)
weight = weather_file_df["Humidity"].astype(float)
humidity_max = weather_file_df["Humidity"].max().astype(float)
humidity_max

100.0

In [5]:
fig = gmaps.figure(zoom_level=2, center=(29,114))

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=humidity_max, point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather_df = weather_file_df.loc[((weather_file_df["Max Temp"] > 75) & (weather_file_df["Max Temp"] < 85) &
                                  ((weather_file_df["Cloudiness"] < 50) & (weather_file_df["Cloudiness"] > 15)) &
                                   (weather_file_df["Wind Speed"] < 10) & (weather_file_df["Humidity"] < 75))]

ideal_weather_df.dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,Norman Wells,65.2820,-126.8329,77.04,22,40,5.75,CA,1624067722
60,Isangel,-19.5500,169.2667,84.31,74,20,5.75,VU,1624067730
176,Butaritari,3.0707,172.7902,82.92,70,25,5.28,KI,1624067576
303,Charter Township of Clinton,42.5870,-82.9199,78.69,74,20,8.05,US,1624067706
340,Semey,50.4111,80.2275,77.22,36,40,8.95,KZ,1624067935
455,Kysyl-Syr,63.8986,122.7617,75.90,51,24,5.95,RU,1624068017


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df.loc[:, ["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Country,Hotel Name
51,Norman Wells,65.2820,-126.8329,CA,
60,Isangel,-19.5500,169.2667,VU,
176,Butaritari,3.0707,172.7902,KI,
303,Charter Township of Clinton,42.5870,-82.9199,US,
340,Semey,50.4111,80.2275,KZ,
455,Kysyl-Syr,63.8986,122.7617,RU,


In [8]:
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_detail = requests.get(base_url, params=params)
    
    hotel_detail = hotel_detail.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_detail["results"][0]["name"]
            
    except:
        print(f"No hotel found within these parameteres for {city_name}... skipping") 
    

No hotel found within these parameteres for Kysyl-Syr... skipping


In [9]:
hotel_df

,City,Lat,Lng,Country,Hotel Name
51,Norman Wells,65.2820,-126.8329,CA,Heritage Hotel
60,Isangel,-19.5500,169.2667,VU,Tanna Lodge
176,Butaritari,3.0707,172.7902,KI,Isles Sunset Lodge
303,Charter Township of Clinton,42.5870,-82.9199,US,Hampton Inn Detroit/Roseville
340,Semey,50.4111,80.2275,KZ,Grand Hotel Semey
455,Kysyl-Syr,63.8986,122.7617,RU,


In [10]:
hotel_name = hotel_df["Hotel Name"].tolist()

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))